In [1]:
!pip install bs4

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [3]:
SELF_HELP_BOOKS_URL = "https://www.amazon.in/s?k=self+help+books&crid=2GXFGYS0COZNM&sprefix=self+help+books%2Caps%2C241&ref=nb_sb_noss_1"
ROMANCE_BOOKS_URL = "https://www.amazon.in/s?k=romance+books&crid=2HI1H7TXQ3DE9&sprefix=romance+book%2Caps%2C227&ref=nb_sb_noss_1"
BIOGRAPHY_BOOKS_URL = "https://www.amazon.in/s?k=BIOGRAPHY+books&crid=YDJZWTELQ63V&sprefix=biography+books%2Caps%2C489&ref=nb_sb_noss_1"
FANTASY_BOOKS_URL = "https://www.amazon.in/s?k=fantasy+books&crid=2QEKSB5TXOA72&sprefix=%2Caps%2C272&ref=nb_sb_ss_recent_3_0_recent"
SCIENCE_FICTION_BOOKS_URL = "https://www.amazon.in/s?k=science+fiction+books&crid=2SS6L112PLCIM&sprefix=science+fiction+books+%2Caps%2C242&ref=nb_sb_noss_2"
THRILLER_BOOKS_URL = "https://www.amazon.in/s?k=THRILLER+books&crid=2SY0XWAW86BB7&sprefix=thriller+books%2Caps%2C223&ref=nb_sb_noss_1"
HISTORICAL_FICTION_BOOKS_URL = "https://www.amazon.in/s?k=historical+fiction+books&crid=2UFA2NCLTLETM&sprefix=HISTORICAL+FIC%2Caps%2C227&ref=nb_sb_ss_mvt-t11-ranker_1_14"
HORROR_BOOKS_URL ="https://www.amazon.in/s?k=horror+books&crid=2S9WKKUTEA7QP&sprefix=horror+books%2Caps%2C284&ref=nb_sb_noss_2"
EDUCATION_BOOKS_URL = "https://www.amazon.in/s?k=coding+books&crid=SWPMY2L2TEP5&sprefix=coo%2Caps%2C875&ref=nb_sb_noss_2"

In [4]:
# Headers for request
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5'
}


In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from urllib.parse import urljoin
import re

# URLs for different book genres (simplified URLs)
BOOK_URLS = {
    "Self Help": "https://www.amazon.in/s?k=self+help+books",
    "Romance": "https://www.amazon.in/s?k=romance+books",
    "Biography": "https://www.amazon.in/s?k=biography+books",
    "Fantasy": "https://www.amazon.in/s?k=fantasy+books",
    "Science Fiction": "https://www.amazon.in/s?k=science+fiction+books",
    "Thriller": "https://www.amazon.in/s?k=thriller+books",
    "Historical Fiction": "https://www.amazon.in/s?k=historical+fiction+books",
    "Horror": "https://www.amazon.in/s?k=horror+books",
    "Programming": "https://www.amazon.in/s?k=programming+books",

    # Additional genres
    "Mystery": "https://www.amazon.in/s?k=mystery+books",
    "Young Adult": "https://www.amazon.in/s?k=young+adult+books",
    "Children's Books": "https://www.amazon.in/s?k=children+books",
    "Comics & Graphic Novels": "https://www.amazon.in/s?k=comics+graphic+novels",
    "Poetry": "https://www.amazon.in/s?k=poetry+books",
    "Spirituality": "https://www.amazon.in/s?k=spiritual+books",
    "Cookbooks": "https://www.amazon.in/s?k=cookbooks",
    "Travel": "https://www.amazon.in/s?k=travel+books",
    "Business": "https://www.amazon.in/s?k=business+books",
    "Health & Fitness": "https://www.amazon.in/s?k=health+fitness+books"
}


# Multiple headers to rotate through
HEADERS_LIST = [
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
        'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
    },
    {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    },
    {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }
]

def extract_book_data(soup, genre):
    """Extract book information from Amazon search results"""
    books = []

    # Debug: Print some of the HTML structure
    print(f"Checking HTML structure for {genre}...")

    # Try multiple selectors for Amazon's changing structure
    selectors = [
        'div[data-component-type="s-search-result"]',
        'div[data-asin]',
        'div.s-result-item',
        'div.sg-col-inner',
        'div.s-widget-container'
    ]

    book_containers = []
    for selector in selectors:
        containers = soup.select(selector)
        if containers:
            book_containers = containers
            print(f"Found {len(containers)} containers using selector: {selector}")
            break

    if not book_containers:
        print(f"No book containers found for {genre}")
        # Save HTML for debugging
        with open(f'debug_{genre.replace(" ", "_")}.html', 'w', encoding='utf-8') as f:
            f.write(str(soup))
        return books

    for i, container in enumerate(book_containers[:20]):  # Limit to first 20 results
        try:
            book_data = {'genre': genre}

            # Extract title - try multiple selectors
            title_selectors = [
                'h2.a-size-mini span',
                'h2.a-size-mini a span',
                'h2 a span',
                'h2 span',
                '.a-size-medium.a-color-base',
                '.a-size-base-plus'
            ]

            for selector in title_selectors:
                title_elem = container.select_one(selector)
                if title_elem and title_elem.get_text(strip=True):
                    book_data['title'] = title_elem.get_text(strip=True)
                    break

            # Extract link
            link_elem = container.select_one('h2 a')
            if link_elem and link_elem.get('href'):
                book_data['url'] = urljoin('https://www.amazon.in', link_elem.get('href'))

            # Extract author - try multiple selectors
            author_selectors = [
                '.a-size-base.a-link-normal',
                '.a-size-base[href*="author"]',
                'span.a-size-base',
                '.s-link-style .a-size-base'
            ]

            for selector in author_selectors:
                author_elem = container.select_one(selector)
                if author_elem and author_elem.get_text(strip=True):
                    author_text = author_elem.get_text(strip=True)
                    if not any(word in author_text.lower() for word in ['kindle', 'paperback', 'hardcover', 'edition']):
                        book_data['author'] = author_text
                        break

            # Extract rating
            rating_elem = container.select_one('.a-icon-alt')
            if rating_elem:
                rating_text = rating_elem.get_text(strip=True)
                rating_match = re.search(r'(\d+\.?\d*) out of 5', rating_text)
                if rating_match:
                    book_data['rating'] = rating_match.group(1)

            # Extract number of reviews
            review_selectors = [
                'span.a-size-base',
                '.a-link-normal span.a-size-base'
            ]

            for selector in review_selectors:
                reviews_elems = container.select(selector)
                for elem in reviews_elems:
                    text = elem.get_text(strip=True).replace(',', '')
                    if text.isdigit():
                        book_data['reviews_count'] = elem.get_text(strip=True)
                        break
                if 'reviews_count' in book_data:
                    break

            # Extract price
            price_selectors = [
                '.a-price-whole',
                '.a-price .a-offscreen',
                '.a-price-symbol + .a-price-whole',
                '.a-price-range .a-price .a-offscreen'
            ]

            for selector in price_selectors:
                price_elem = container.select_one(selector)
                if price_elem:
                    price_text = price_elem.get_text(strip=True)
                    if price_text and price_text != '':
                        book_data['price'] = price_text
                        break

            # Extract image URL
            img_elem = container.select_one('img')
            if img_elem and img_elem.get('src'):
                book_data['image_url'] = img_elem.get('src')

            # Extract format
            format_selectors = [
                '.a-size-base-plus',
                '.a-size-base .a-color-secondary',
                'span.a-size-base.a-color-secondary'
            ]

            for selector in format_selectors:
                format_elem = container.select_one(selector)
                if format_elem:
                    format_text = format_elem.get_text(strip=True)
                    if any(word in format_text.lower() for word in ['kindle', 'paperback', 'hardcover']):
                        book_data['format'] = format_text
                        break

            # Only add books with at least a title
            if 'title' in book_data and book_data['title']:
                books.append(book_data)
                print(f"Extracted book {i+1}: {book_data.get('title', 'Unknown')}")

        except Exception as e:
            print(f"Error extracting book {i+1}: {e}")
            continue

    return books

def scrape_books():
    """Main function to scrape books from all genres"""
    all_books = []
    session = requests.Session()

    for genre, url in BOOK_URLS.items():
        print(f"\nScraping {genre} books...")

        try:
            # Rotate headers and add random delay
            headers = random.choice(HEADERS_LIST)
            time.sleep(random.uniform(2, 5))

            # Make request with session
            response = session.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            print(f"Response status: {response.status_code}")
            print(f"Response size: {len(response.content)} bytes")

            # Check if we got blocked
            if "robot" in response.text.lower() or "captcha" in response.text.lower():
                print(f"Detected blocking for {genre}. Trying different approach...")
                time.sleep(random.uniform(5, 10))

                # Try with different headers
                headers = random.choice(HEADERS_LIST)
                response = session.get(url, headers=headers, timeout=10)

            soup = BeautifulSoup(response.content, 'html.parser')

            # Check if we have search results
            if "did not match any products" in response.text:
                print(f"No products found for {genre}")
                continue

            books = extract_book_data(soup, genre)
            all_books.extend(books)
            print(f"Successfully extracted {len(books)} books from {genre}")

        except requests.RequestException as e:
            print(f"Request error for {genre}: {e}")
            continue
        except Exception as e:
            print(f"Error processing {genre}: {e}")
            continue

    return all_books

def save_to_csv(books, filename='amazon_books.csv'):
    """Save book data to CSV file"""
    if not books:
        print("No books found to save.")
        return

    # Get all possible field names
    fieldnames = set()
    for book in books:
        fieldnames.update(book.keys())

    fieldnames = sorted(list(fieldnames))

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(books)

    print(f"Saved {len(books)} books to {filename}")

def main():
    """Main execution function"""
    print("Starting Amazon book scraper...")

    # Scrape books
    books = scrape_books()

    # Save to CSV
    save_to_csv(books)

    # Print summary
    print(f"\nScraping completed!")
    print(f"Total books scraped: {len(books)}")

    # Print genre breakdown
    genre_counts = {}
    for book in books:
        genre = book.get('genre', 'Unknown')
        genre_counts[genre] = genre_counts.get(genre, 0) + 1

    print("\nBooks per genre:")
    for genre, count in genre_counts.items():
        print(f"  {genre}: {count}")

if __name__ == "__main__":
    main()

Starting Amazon book scraper...

Scraping Self Help books...
Response status: 200
Response size: 1033048 bytes
Detected blocking for Self Help. Trying different approach...
Checking HTML structure for Self Help...
Found 16 containers using selector: div[data-component-type="s-search-result"]
Extracted book 1: Don't Believe Everything You Think (English)
Extracted book 2: DEEP WORK: RULES FOR FOCUSED SUCCESS IN A DISTRACTED WORLD
Extracted book 3: Dopamine Detox: A Short Guide to Remove Distractions and Get Your Brain to Do Hard Things
Extracted book 4: The Mountain Is You: Transforming Self-Sabotage Into Self-Mastery (English)
Extracted book 5: Attitude is Everything change your attitude change your life!
Extracted book 6: The Art of Letting Go by Nick Trenton | A Self-Help Guide to Emotional Freedom, Inner Peace & Overcoming Anxiety | Learn to Heal, Grow & Move On from the Past
Extracted book 7: The Power of Your Subconscious Mind: Original Classic Edition | Premium Paperback
Extracte